In [ ]:
# default_exp io.core

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import argparse
import logging
import sys
import shutil
from os import listdir, makedirs
from os.path import join, isfile, isdir, splitext

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

DEFAULT_IMAGE_EXTENSIONS = ['.jpg']
FOLDER_FILTER = ['.DS_Store']

In [ ]:
# export

logger = logging.getLogger(__name__)

# IO Core functions

> Helper functions for IO operations.

In [ ]:
# export


def scan_files(folder, file_extensions=None):
    """
    Scan the folder for files and filter files by file extension.
    If the optional `file_extension` is not set, **.jpg** as file extension is used by default.
    `folder`: the folder to scan for files
    `file_extensions`: the allowed file extensions
    return: the file path list
    """
    if file_extensions is None:
        file_extensions = DEFAULT_IMAGE_EXTENSIONS

    results = []
    if isdir(folder):
        files = listdir(folder)
        for file in files:
            file_path = join(folder, file)
            filename, file_extension = splitext(file_path)
            if isfile(file_path) and (file_extension in file_extensions):
                results.append(file_path)
            elif isdir(file_path):
                results.extend(scan_files(file_path, file_extensions))
    return results

In [ ]:
# export


def scan_path(path):
    """
    Scan a folder and subfolders until the file level.
    `path`: the current directory to parse for folders
    return: a list of folders prefixed with base directory
    """
    has_sub_folders = False
    results = []
    names = listdir(path)
    for name in names:
        subfolder_path = join(path, name)
        if isdir(subfolder_path) and name not in FOLDER_FILTER:
            has_sub_folders = True
            results = results + scan_path(subfolder_path)
    if not has_sub_folders:
        results.append(path)
    return results

In [ ]:
# export


def delete_folder(path):
    """
    Deletes a Folder, if exist.
    `path`: the folder path including the folder name
    return: the created folder path
    """
    if isdir(path):
        shutil.rmtree(path)
    return path

In [ ]:
# export


def create_folder(path, clear=False):
    """
    Creates a Folder, if not exist.
    `path`: the folder path including the folder name
    `clear`: wherether or not the created folder should be empty
    return: the created folder path
    """

    if clear:
        # remove previous data-set folder, if exist to clear the contents
        delete_folder(path)

    if not isdir(path):
        makedirs(path)

    return path

## Helper Methods

In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.

    :param logging_level: The logging level to use.
    """
    logger.setLevel(logging_level)

    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging_level)

    logger.addHandler(handler)

## Run from command line

To run the data-set builder from command line, use the following command:
`python -m mlcore.via.converter [parameters]`

The following parameters are supported:
- `[annotation]`: The path to the VIA annotation file (e.g.: *imagesets/segmentation/car_damage/via_region_data.json*)

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    # for direct shell execution
    parser = argparse.ArgumentParser()
    parser.add_argument("annotation",
                        help="The path to the VIA annotation file.")

    args = parser.parse_args()
